In [ ]:
# Math125 Final Project



# by Xianmai(Chris) Liang and James Ma





In [ ]:
import numpy as np
import pandas as pd
from numpy import linalg as LA
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import euclidean_distances
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)
import csv
import os
import sklearn.metrics.pairwise as pdist
from sklearn.metrics.pairwise import euclidean_distances


<ipython-input-2-a779fb4e0174>:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from heapq import heapify, heappush, heappop

import numpy as np
from numpy import array
import scipy.sparse as sp
from sklearn.preprocessing import normalize
from scipy.sparse import diags
from scipy.sparse import csr_matrix

import time
t_s = time.time()

import sklearn.metrics.pairwise as pdist
from sklearn.metrics.pairwise import euclidean_distances
import gc

gc.collect()
import random

In [ ]:
def getLabDic(lables):
    """input a list of lables
    return a dictionary mapping labels to 0,1,2,3...n-1
    """
    Labs=list(set(lables))
    print(Labs)
    n=len(Labs)
    labdic={}
    for i in range(n):
        labdic[Labs[i]]=i
    for k,v in labdic.items():
        print(str(k)+"--->"+str(v))
    return labdic

def getdatafromFile(filenm):
    """ this function uses the input filename
      assume each line: label +\t+ line
      return list of labels, list of lines
    """
    dat=[]
    Y=[]
    openf=open(filenm, 'r', encoding="latin1")
    for i, ln in enumerate(openf):
        ln2=ln.split('\t')
        Y.append(ln2[0])
        dat.append(ln2[1])
    openf.close()
    return (dat,Y)

In [ ]:
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
test_r8fn='r8-test-all-terms.txt'
train_r8fn='r8-train-all-terms.txt'
test_20ngfn='20ng-test-all-terms.txt'
train_20ngfn='20ng-train-all-terms.txt'
# train_ohfn='train_ohsumed_by_line.txt'
# test_ohfn='test_ohsumed_by_line.txt'


In [ ]:
(traindat,trainY)=getdatafromFile(train_r8fn)
(testdat, testY)=getdatafromFile(test_r8fn)
r8Labdic=getLabDic(trainY)
train_lab=[r8Labdic[e] for e in trainY]
test_lab=[r8Labdic[e] for e in testY]

['crude', 'interest', 'trade', 'acq', 'grain', 'money-fx', 'ship', 'earn']
crude--->0
interest--->1
trade--->2
acq--->3
grain--->4
money-fx--->5
ship--->6
earn--->7


In [ ]:
print(len(train_lab))
print(len(test_lab))
train_lab=np.asarray(train_lab)
test_lab=np.asarray(test_lab)

5485
2189


In [ ]:
r8LabdicRev={}
for k,v in r8Labdic.items():
    r8LabdicRev[v]=k

In [ ]:
docs=['I like Brandeis', 'machine learning', 'I work at Brandeis','machine learning is awesome']

In [ ]:
vect = CountVectorizer(ngram_range=(1,1), stop_words=None)
N=len(docs)
print(f"there are { N } docs.")
tfx=vect.fit_transform(docs)
print(tfx.shape)
vocabs=vect.get_feature_names_out()
print(f"length of vocab={len(vocabs)}")
dfs=csr_matrix(((tfx>0)*1).sum(axis=0)).toarray()[0]

there are 4 docs.
(4, 8)
length of vocab=8


In [ ]:
dfs

array([1, 1, 2, 1, 2, 1, 2, 1])

In [ ]:
vocabs

array(['at', 'awesome', 'brandeis', 'is', 'learning', 'like', 'machine',
       'work'], dtype=object)

In [ ]:
tfx.toarray()

array([[0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 0, 0, 0, 1],
       [0, 1, 0, 1, 1, 0, 1, 0]])

In [ ]:
tfx2=normalize(tfx, norm='l1',axis=1)

In [ ]:
tfx2.toarray()

array([[0.        , 0.        , 0.5       , 0.        , 0.        ,
        0.5       , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.5       ,
        0.        , 0.5       , 0.        ],
       [0.33333333, 0.        , 0.33333333, 0.        , 0.        ,
        0.        , 0.        , 0.33333333],
       [0.        , 0.25      , 0.        , 0.25      , 0.25      ,
        0.        , 0.25      , 0.        ]])

In [ ]:
dfs=csr_matrix(((tfx>0)*1).sum(axis=0)).toarray()[0]

In [ ]:
dfs

array([1, 1, 2, 1, 2, 1, 2, 1])

In [ ]:
def tfidf(traindocs, stopwordlist=None):
    vect = CountVectorizer(ngram_range=(1,1), stop_words=stopwordlist)
    N=len(traindocs)
    # print(f"there are { N } docs.")
    tfx=vect.fit_transform(traindocs)
    # print(tfx.shape)
    vocabs=vect.get_feature_names_out()
    # print(f"length of vocab={len(vocabs)}")
    dfs=csr_matrix(((tfx>0)*1).sum(axis=0)).toarray()[0]
    # print("shape of dfs:"+str(dfs.shape))
    # print("len of dfs: "+str(len(dfs)))

    vocabs=vect.get_feature_names_out()
    # print(f"length of vocab={len(vocabs)}")

    tfx=normalize(tfx, norm='l1',axis=1)
    # print("shape of tfx1: "+str(tfx.shape))
    # print("computing tfidf ... ")
    idfs=[1+np.log(N/(1+d)) for d in dfs]
    idfs_diag=diags(idfs)
    tfidf=tfx.dot(idfs_diag)
    return tfidf

In [ ]:
tfidf(traindat)

<5485x19956 sparse matrix of type '<class 'numpy.float64'>'
	with 320397 stored elements in Compressed Sparse Row format>

In [ ]:
def evaluate(testdocs, testlabs):
    predictionlabs=predict(testdocs)
    #prediction=np.asarray(prediction)
    count=0; ntest=len(predictionlabs)
    for i in range(ntest):
        if predictionlabs[i]==testlabs[i]:
            count=count+1
    test_error = 1 - count/ntest
    return test_error


def KNNpredict(neighbor_classes, C, cross_val=False):
    # Make sure all classes are considered
    labels = np.concatenate((neighbor_classes, list(range(C))))
    # Find class frequency among neighbors
    weights = np.unique(labels, return_counts=True)[1]
    # Find most popular class
    prediction = np.argmax(weights)
    nn=len(neighbor_classes)
    # If most popular class is ambiguous try with fewer neighbors; else return
    if sum(weights[prediction] == weights) > 1 and nn>2:
        return KNNpredict(neighbor_classes[:-2], C)
    else:
        return prediction



def knnClassify(y_train, dist, n_neighbors=7):
    ## y_train: training docs labels; dist: testdocs vs traindocs pairwise distance matrix
    # Number of all different classes
    #n_classes = len(np.unique(y_train))
    prediction = []
    ntestsample=dist.shape[0]
    for i in range(ntestsample):
        doc_to_train=dist[i,:]
        # Find indices of n_neighbors closest documents
        rank = np.argsort(doc_to_train)[:n_neighbors]
        # Make prediction based on most popular class among neighbors
        prediction.append(KNNpredict(y_train[rank], n_classes))
    return prediction


In [109]:
# logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

vect = TfidfVectorizer(ngram_range=(1, 1), stop_words=None)
X_train_tfidf = vect.fit_transform(traindat)

X_test_tfidf = vect.transform(testdat)

lr_model = LogisticRegression()
lr_model.fit(X_train_tfidf, train_lab)

test_predictions = lr_model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(test_lab, test_predictions))
print(classification_report(test_lab, test_predictions))

Accuracy: 0.9488350845134764
              precision    recall  f1-score   support

           0       0.90      0.99      0.94       696
           1       0.84      0.64      0.73        87
           2       0.98      0.89      0.94       121
           3       1.00      0.50      0.67        10
           4       0.92      0.93      0.93        75
           5       0.95      0.74      0.83        81
           6       0.99      0.99      0.99      1083
           7       1.00      0.56      0.71        36

    accuracy                           0.95      2189
   macro avg       0.95      0.78      0.84      2189
weighted avg       0.95      0.95      0.95      2189



In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=5)

knn_model.fit(X_train_tfidf, train_lab)

test_predictions = knn_model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(test_lab, test_predictions))
print(classification_report(test_lab, test_predictions))


Accuracy: 0.8835084513476473
              precision    recall  f1-score   support

           0       0.97      0.75      0.85       696
           1       0.80      0.90      0.84        87
           2       0.90      0.92      0.91       121
           3       0.88      0.70      0.78        10
           4       0.70      0.97      0.81        75
           5       0.82      0.73      0.77        81
           6       0.87      0.98      0.92      1083
           7       0.85      0.61      0.71        36

    accuracy                           0.88      2189
   macro avg       0.85      0.82      0.82      2189
weighted avg       0.89      0.88      0.88      2189



In [110]:
# Random forest
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()

rf_model.fit(X_train_tfidf, train_lab)


test_predictions = rf_model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(test_lab, test_predictions))
print(classification_report(test_lab, test_predictions))

Accuracy: 0.9168570123343993
              precision    recall  f1-score   support

           0       0.83      0.98      0.90       696
           1       0.76      0.47      0.58        87
           2       1.00      0.74      0.85       121
           3       1.00      0.40      0.57        10
           4       0.84      0.85      0.85        75
           5       0.96      0.60      0.74        81
           6       0.98      0.99      0.99      1083
           7       1.00      0.17      0.29        36

    accuracy                           0.92      2189
   macro avg       0.92      0.65      0.72      2189
weighted avg       0.92      0.92      0.91      2189



In [111]:
# SVM
from sklearn.svm import SVC

svm_model = SVC()

svm_model.fit(X_train_tfidf, train_lab)

test_predictions = svm_model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(test_lab, test_predictions))
print(classification_report(test_lab, test_predictions))

Accuracy: 0.9470077661032434
              precision    recall  f1-score   support

           0       0.89      0.99      0.94       696
           1       0.86      0.63      0.73        87
           2       0.99      0.88      0.93       121
           3       1.00      0.60      0.75        10
           4       0.93      0.92      0.93        75
           5       0.94      0.77      0.84        81
           6       0.99      0.99      0.99      1083
           7       1.00      0.44      0.62        36

    accuracy                           0.95      2189
   macro avg       0.95      0.78      0.84      2189
weighted avg       0.95      0.95      0.94      2189



In [113]:
# Naive Bayes

from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()

nb_model.fit(X_train_tfidf, train_lab)

test_predictions = nb_model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(test_lab, test_predictions))
print(classification_report(test_lab, test_predictions))


Accuracy: 0.8195523069894929
              precision    recall  f1-score   support

           0       0.72      0.96      0.83       696
           1       0.80      0.05      0.09        87
           2       0.97      0.31      0.47       121
           3       0.00      0.00      0.00        10
           4       1.00      0.12      0.21        75
           5       1.00      0.06      0.12        81
           6       0.89      0.99      0.93      1083
           7       0.00      0.00      0.00        36

    accuracy                           0.82      2189
   macro avg       0.67      0.31      0.33      2189
weighted avg       0.83      0.82      0.77      2189



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [119]:
# Decision tree
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(max_depth=10)

dt_model.fit(X_train_tfidf, train_lab)

test_predictions = dt_model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(test_lab, test_predictions))
print(classification_report(test_lab, test_predictions))


Accuracy: 0.8835084513476473
              precision    recall  f1-score   support

           0       0.80      0.91      0.85       696
           1       0.67      0.25      0.37        87
           2       0.88      0.89      0.89       121
           3       1.00      0.30      0.46        10
           4       0.92      0.80      0.86        75
           5       0.81      0.62      0.70        81
           6       0.95      0.96      0.96      1083
           7       1.00      0.33      0.50        36

    accuracy                           0.88      2189
   macro avg       0.88      0.63      0.70      2189
weighted avg       0.88      0.88      0.87      2189



In [121]:
# tried to implement LSTM but really bad accuracy 

# pip install tensorflow
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences


In [122]:
# tokenizer = Tokenizer(num_words=5000)
# tokenizer.fit_on_texts(traindat)

# X_train_seq = tokenizer.texts_to_sequences(traindat)
# X_test_seq = tokenizer.texts_to_sequences(testdat)

# max_seq_length = 100
# X_train = pad_sequences(X_train_seq, maxlen=max_seq_length)
# X_test = pad_sequences(X_test_seq, maxlen=max_seq_length)

# model = Sequential()
# model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_seq_length))
# model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# model.fit(X_train, train_lab, batch_size=32, epochs=5)
# loss, accuracy = model.evaluate(X_test, test_lab)
# print("Test Accuracy:", accuracy)


Epoch 1/5
172/172 [==============================] - 64s 309ms/step - loss: -58.3485 - accuracy: 0.0379
Epoch 2/5
172/172 [==============================] - 58s 337ms/step - loss: -123.8005 - accuracy: 0.0376
Epoch 3/5
172/172 [==============================] - 48s 277ms/step - loss: -183.0952 - accuracy: 0.0376
Epoch 4/5
172/172 [==============================] - 47s 271ms/step - loss: -242.8777 - accuracy: 0.0376
Epoch 5/5
69/69 [==============================] - 4s 38ms/step - loss: -310.8559 - accuracy: 0.0397
Test Accuracy: 0.03974417597055435


In [123]:
#Same for RNN

# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, Dropout
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences


In [124]:
# tokenizer = Tokenizer(num_words=5000)
# tokenizer.fit_on_texts(traindat)

# X_train_seq = tokenizer.texts_to_sequences(traindat)
# X_test_seq = tokenizer.texts_to_sequences(testdat)

# max_seq_length = 100  # Define the max sequence length
# X_train = pad_sequences(X_train_seq, maxlen=max_seq_length)
# X_test = pad_sequences(X_test_seq, maxlen=max_seq_length)


In [125]:
# model = Sequential()
# model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_seq_length))
# model.add(SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [126]:
# model.fit(X_train, train_lab, batch_size=32, epochs=5)
# loss, accuracy = model.evaluate(X_test, test_lab)
# print("Test Accuracy:", accuracy)


Epoch 1/5
172/172 [==============================] - 14s 67ms/step - loss: -49.7079 - accuracy: 0.0423
Epoch 2/5
172/172 [==============================] - 11s 66ms/step - loss: -126.3939 - accuracy: 0.0376
Epoch 3/5
172/172 [==============================] - 11s 61ms/step - loss: -190.8492 - accuracy: 0.0379
Epoch 4/5
172/172 [==============================] - 11s 64ms/step - loss: -253.1404 - accuracy: 0.0385
Epoch 5/5
69/69 [==============================] - 1s 11ms/step - loss: -322.3336 - accuracy: 0.0411
Test Accuracy: 0.04111466556787491
